# A Simple MNIST model

A digit recognition model implemented with Tensorflow and Python.

Adapted from: https://github.com/sethjuarez/SimpleMNIST

## Requirements

* Anaconda
* Tensorflow
* Azure Machine Learning

## Setup instructions

See: [Configure a development environment for Azure Machine Learning](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-environment?WT.mc_id=AILive-workshop-davidsmi)


In [ ]:
import json
import time
import azureml
import logging
from azureml.core.model import Model
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace, Run, Datastore, Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.hyperdrive import *
from azureml.train.dnn import TensorFlow
from azureml.widgets import RunDetails

print("Azure ML SDK Version: ", azureml.core.VERSION)

You will need to provide a `config.json` file in the working folder to provide keys for your Azure ML workspace. See Seup Instructions above for details.

In [ ]:
# use this code to set up config file
#subscription_id ='<SUB_ID>'
#resource_group ='<RESOURCE_GROUP>'
#workspace_name = '<WORKSPACE>'

#try:
#   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#   ws.write_config()
#   print('Workspace configuration succeeded. You are all set!')
#except:
#   print('Workspace not found. TOO MANY ISSUES!!!')
ws = Workspace.from_config()

# Compute Environment

In [ ]:
cluster = 'sauron'
try:
    compute = ComputeTarget(workspace=ws, name=cluster)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', min_nodes=1, max_nodes=6)
    compute = ComputeTarget.create(ws, cluster, compute_config)
    compute.wait_for_completion(show_output=True)
    

# Run Experiment

In [ ]:
# Create and run experiment
mnist = Experiment(ws, 'dms-simplemnist')
estimator = TensorFlow(source_directory='.',
                       compute_target=compute,
                       entry_script='train.py',
                       use_gpu=True)

run = mnist.submit(estimator)

In [ ]:
run

In [ ]:
RunDetails(run).show()

# Manage Data

In [ ]:
ds = ws.get_default_datastore()
mnist_data = ds.upload(src_dir = 'data', target_path = 'mnist', show_progress = True)

In [ ]:
# run the same was as above
script_params={
    '--data': mnist_data.as_mount(),
}

# Create and run experiment
mnist = Experiment(ws, 'simplemnist')
estimator = TensorFlow(source_directory='.',
                       compute_target=compute,
                       entry_script='train.py',
                       script_params=script_params,
                       use_gpu=True)

run = mnist.submit(estimator)
RunDetails(run).show()

# Optimize hyperparameters

In [ ]:
# same as above but increase the max_steps and remove the parameters
script_params={
    '--data': mnist_data,
    '--epochs': 100
}

mnist = Experiment(ws, 'simplemnist')
estimator = TensorFlow(source_directory='.',
                       compute_target=compute,
                       entry_script='train.py',
                       script_params=script_params,
                       use_gpu=True)

## hyperparameter search

In [ ]:
ps = RandomParameterSampling(
    {
        '--lr': loguniform(-15, -3),
        '--batch': choice(16, 32, 64, 128, 512)
    }
)

early_termination_policy = BanditPolicy(slack_factor = 0.15, evaluation_interval=2)

hyperdrive_run_config = HyperDriveRunConfig(estimator = estimator, 
                                            hyperparameter_sampling = ps, 
                                            policy = early_termination_policy,
                                            primary_metric_name = "accuracy",
                                            primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                            max_total_runs = 20,
                                            max_concurrent_runs = 5)

hd_run = mnist.submit(hyperdrive_run_config)

RunDetails(hd_run).show()

# Deploy best model

In [ ]:
best = hd_run.get_best_run_by_primary_metric()
best.get_file_names()

In [ ]:
best.download_file(name='outputs/digits.pb', output_file_path='outputs')
model_file = 'outputs/digits.pb'
model = Model.register(ws, model_name='SimpleMNIST', model_path=model_file, 
                       description='Simple MNIST model extracted from hyperparamter optimization run')

## Create Conda Deps

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
myenv = CondaDependencies()

myenv.add_pip_package('numpy')
myenv.add_tensorflow_pip_package()

with open('simplemnist.yml','w') as f:
    print('Writing out {}'.format('simplemnist.yml'))
    f.write(myenv.serialize_to_string())
    print('Done!')

## Create Image

In [ ]:
model = ws.models['SimpleMNIST']

In [ ]:
from azureml.core.image import ContainerImage, Image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                    runtime="python", 
                                    conda_file="simplemnist.yml")


image = Image.create(ws, 'simplemnist', [model], image_config)
image.wait_for_creation(show_output=True)

In [ ]:
image.image_build_log_uri

## Deploy Service

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               description='simple MNIST digit detection')
service = Webservice.deploy_from_image(workspace=ws, 
                                       image=image, 
                                       name='simplemnist-svc', 
                                       deployment_config=aciconfig)
service.wait_for_deployment(show_output=True)

In [ ]:
service.scoring_uri

In [ ]:
with open('deploy.log','w') as f:
    f.write(service.get_logs())

# Automatic Machine Learning

In [ ]:
automl_settings = {
    "name": "AutoML_Demo_Experiment_{0}".format(time.time()),
    "iteration_timeout_minutes": 5,
    "iterations": 20,
    "n_cross_validations": 5,
    "primary_metric": 'AUC_weighted',
    "preprocess": False,
    "max_concurrent_iterations": 10,
    "verbosity": logging.INFO
}

automl_config = AutoMLConfig(task='classification',
                             debug_log='automl_errors.log',
                             path='.',
                             compute_target = compute,
                             data_script='./get_data.py',
                             **automl_settings)

In [ ]:
experiment=Experiment(ws, 'simplemnist')
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()